In [1]:
import numpy as onp

from jax.config import config ; config.update('jax_enable_x64', True)
# config.update('jax_disable_jit', True)

import jax.numpy as jnp
from jax import random
from jax import jit, grad, xla_computation, make_jaxpr
from jax import lax

import time

from jax_md import space
from jax_md import smap
from jax_md import energy
from jax_md import quantity
from jax_md import simulate
from jax_md import partition

import matplotlib
import matplotlib.pyplot as plt

import ase

import jax.profiler


## System

In [2]:
from uf3.representation import bspline
from uf3.data import composition
from uf3.regression import least_squares

element_list = ['W']
degree = 2

chemical_system = composition.ChemicalSystem(element_list=element_list,
                                             degree=degree)

r_min_map = {('W', 'W'): 1.5,
            }
r_max_map = {('W', 'W'): 5.5,
            }
resolution_map = {('W', 'W'): 25,
                 }
trailing_trim = 3

bspline_config = bspline.BSplineBasis(chemical_system,
                                      r_min_map=r_min_map,
                                      r_max_map=r_max_map,
                                      resolution_map=resolution_map,
                                      trailing_trim=trailing_trim)

model_2_body = least_squares.WeightedLinearModel(bspline_config)

model_2_body.load(filename="../../tungsten_extxyz/model_pair.json")

In [3]:
from uf3.forcefield.calculator import *

element_list = ['W']
degree = 3

chemical_system = composition.ChemicalSystem(element_list=element_list,
                                             degree=degree)

r_min_map = {("W", "W"): 1.5,
             ("W", "W", "W"): [1.5, 1.5, 1.5],
            }
r_max_map = {("W", "W"): 5.5,
             ("W", "W", "W"): [3.5, 3.5, 7.0],
            }
resolution_map = {("W", "W"): 25,
                  ("W", "W", "W"): [5, 5, 10],
                 }
trailing_trim = 3

bspline_config = bspline.BSplineBasis(chemical_system,
                                      r_min_map=r_min_map,
                                      r_max_map=r_max_map,
                                      resolution_map=resolution_map,
                                      trailing_trim=trailing_trim)

model_3_body = least_squares.WeightedLinearModel(bspline_config)

model_3_body.load(filename="../../tungsten_extxyz/model_uf23.json")

In [4]:
Nx = particles_per_side = 10
# Nx = particles_per_side = 50

spacing = np.float32(2.0)
side_length = Nx * spacing

R = onp.stack([np.array(r) for r in np.ndindex(Nx, Nx, Nx)]) * spacing
R = np.array(R, np.float64)

In [5]:
print(f"Number of atoms: {R.shape[0]}")
print(f"Box should be larger than 11 for JAX-MD, is: {side_length}")

Number of atoms: 1000
Box should be larger than 11 for JAX-MD, is: 20.0


In [6]:
pos = R
c = onp.identity(3) * side_length
ase.Atoms('W13824', positions=pos, cell=c, pbc=[True,True,True])
# ase.Atoms('W125000', positions=pos, cell=c, pbc=[True,True,True])

ValueError: Array "positions" has wrong length: 125000 != 13824.

In [6]:
displacement, shift = space.periodic(side_length)
format = partition.Dense
R = jnp.asarray(R)

In [7]:
calc = UFCalculator(model_3_body)

ndspline2 = calc.pair_potentials[('W','W')]
ndspline3 = calc.trio_potentials[('W','W','W')]

coefficients2 = jnp.asarray(ndspline2.coefficients)
coefficients2 = coefficients2[:,0]

knots2 = ndspline2.knots
knots2 = [jnp.asarray(i) for i in knots2]

coefficients3 = jnp.asarray(ndspline3.coefficients)
coefficients3 = coefficients3[:,:,:,0]

knots3 = ndspline3.knots
knots3 = [jnp.asarray(i) for i in knots3]

## Lennard Jones

In [8]:
from jax_md.energy import lennard_jones_neighbor_list
from jax_md import partition

In [9]:
neighbor_fn, energy_fn = lennard_jones_neighbor_list(displacement, side_length, r_cutoff=5.5, format=partition.Dense)

In [10]:
nbrs = neighbor_fn.allocate(R)
energy_fn = jit(energy_fn, backend='cpu')

In [11]:
nbrs.idx.shape

(1000, 115)

In [12]:
print(energy_fn(R, nbrs))

-239.80404120801424


In [13]:
displacement, shift = space.periodic(side_length)

init_fn, apply_fn = simulate.nve(energy_fn, shift, 1e-3)
state = init_fn(random.PRNGKey(0), R, kT=1e-3, neighbor=nbrs)


def body_fn(i, state):
    state, nbrs = state
    nbrs = nbrs.update(state.position)
    state = apply_fn(state, neighbor=nbrs)
    return state, nbrs


print('Step\tKE\tPE\tTotal Energy\ttime/step')
print('----------------------------------------')
PE = []
KE = []
print_every = 4
step = 0
old_time = time.time_ns()
while step < 40:
    new_state, nbrs = lax.fori_loop(0, 100, body_fn, (state, nbrs))
    if nbrs.did_buffer_overflow:
        print("Neighbor list overflowed, reallocating.")
        nbrs = neighbor_fn.allocate(state.position)
    else:
        state = new_state
        step += 1

    if step % print_every == 0 and step > 0:
        new_time = time.time_ns()
        PE += [energy_fn(state.position, nbrs)]
        KE += [quantity.kinetic_energy(state.velocity)]
        print(
            "{}\t{:.2f}\t{:.2f}\t{:.2f}\t\t{:.2f}ms".format(
                step * print_every,
                KE[-1],
                PE[-1],
                KE[-1] + PE[-1],
                (new_time - old_time) / print_every / 100.0 / 1000000.0,
            )
        )
        old_time = new_time

Step	KE	PE	Total Energy	time/step
----------------------------------------
16	1.83	-240.09	-238.25		30.37ms
32	3.05	-241.30	-238.25		13.42ms
48	6.97	-245.23	-238.25		13.06ms
64	20.73	-258.98	-238.25		13.45ms
80	80.21	-318.47	-238.25		14.41ms
Neighbor list overflowed, reallocating.
96	222.67	-460.93	-238.25		43.82ms
112	380.73	-618.99	-238.26		18.09ms
128	485.20	-723.46	-238.26		17.50ms
144	563.57	-801.82	-238.26		18.05ms
160	625.43	-863.69	-238.25		18.12ms


## Stillinger-Weber

In [8]:
from jax_md.energy import stillinger_weber_neighbor_list
# from jax_md import partition

In [9]:
neighbor_fn, energy_fn =stillinger_weber_neighbor_list(displacement, side_length, cutoff=5.5)

In [10]:
nbrs = neighbor_fn.allocate(R)
energy_fn = jit(energy_fn, backend='cpu')

In [11]:
nbrs.idx.shape

(1000, 115)

In [12]:
print(energy_fn(R, nbrs))

1569003.1157833745


In [ ]:
print(make_jaxpr(energy_fn)(R,nbrs))

In [13]:
print(grad(energy_fn)(R, nbrs))

[[-2.27373675e-13 -5.68434189e-14 -1.17239551e-13]
 [-3.19744231e-13 -1.56319402e-13 -7.81597009e-14]
 [-3.01980663e-13  3.55271368e-14  1.77635684e-14]
 ...
 [ 3.41227047e-13  4.16888746e-14 -5.68434189e-14]
 [ 4.07630527e-13  1.41497924e-13 -9.94759830e-14]
 [ 1.59313535e-13 -5.02931030e-14  6.39488462e-14]]


In [18]:
from jax import vmap
from functools import partial
sigma: float = 2.0951
gamma: float = 1.2
cutoff: float = 3.77118
three_body_fn = partial(_sw_angle_interaction, gamma, sigma, cutoff)
three_body_fn = vmap(vmap(vmap(three_body_fn, (0, None)), (None, 0)))

In [19]:
dR = space.map_product(displacement)(R, R)

In [20]:
c = xla_computation(three_body_fn)(dR,dR)

In [ ]:
print(c.as_hlo_text())

In [ ]:
displacement, shift = space.periodic(side_length)

init_fn, apply_fn = simulate.nve(energy_fn, shift, 1e-3)
state = init_fn(random.PRNGKey(0), R, kT=1e-3, neighbor=nbrs)


def body_fn(i, state):
    state, nbrs = state
    nbrs = nbrs.update(state.position)
    state = apply_fn(state, neighbor=nbrs)
    return state, nbrs


print('Step\tKE\tPE\tTotal Energy\ttime/step')
print('----------------------------------------')
PE = []
KE = []
print_every = 4
step = 0
old_time = time.time_ns()
while step < 40:
    new_state, nbrs = lax.fori_loop(0, 100, body_fn, (state, nbrs))
    # if nbrs.did_buffer_overflow:
    #     print("Neighbor list overflowed, reallocating.")
    #     nbrs = neighbor_fn.allocate(state.position)
    # else:
    state = new_state
    step += 1

    if step % print_every == 0 and step > 0:
        new_time = time.time_ns()
        PE += [energy_fn(state.position, nbrs)]
        KE += [quantity.kinetic_energy(state.velocity)]
        print(
            "{}\t{:.2f}\t{:.2f}\t{:.2f}\t\t{:.2f}ms".format(
                step * print_every,
                KE[-1],
                PE[-1],
                KE[-1] + PE[-1],
                (new_time - old_time) / print_every / 100.0 / 1000000.0,
            )
        )
        old_time = new_time

Step	KE	PE	Total Energy	time/step
----------------------------------------
16	24530.95	1542608.80	1567139.75		844.56ms
32	94254.65	1523366.50	1617621.15		799.78ms


## UF potentials

In [8]:
from uf3.jax.potentials import uf3_neighbor

In [9]:
c = [coefficients2, coefficients3]
k = [knots2, knots3]

In [10]:
neighbor_fn, energy_fn = uf3_neighbor(displacement, side_length, coefficients=c, knots=k, cutoff=5.5)

In [11]:
nbrs = neighbor_fn.allocate(R)
energy_fn = jit(energy_fn, backend='gpu')

In [12]:
# jax.profiler.save_device_memory_profile("potential1.prof")

In [13]:
# Run once for jit
print(energy_fn(R, nbrs))
# print(grad(energy_fn)(R, nbrs))

26811.792926319944


In [14]:
nbrs.idx.shape

(1000, 115)

In [15]:
print(grad(energy_fn)(R, nbrs))
# For #R 13824 with 115 nbrs 
# 54 GB with checkpoint
# 329 GB without checkpoint

# without checkpoint max 1000 R with 115 nbrs at 268ms
# with checkpoint max 3375 R with 115 nbrs at 1.58s


[[ 1.94955163e-13  1.02334807e-13 -2.44589071e-13]
 [ 2.35179931e-13  1.96058447e-13  2.13655482e-13]
 [ 1.08982268e-13  1.33802691e-13 -7.18106130e-14]
 ...
 [-2.60232497e-13 -8.03107580e-14  7.93427823e-14]
 [-2.10068906e-14 -1.51365032e-13  2.01332007e-14]
 [-2.27869942e-13  1.59875585e-13 -9.59197999e-14]]


In [16]:
%%timeit
grad(energy_fn)(R, nbrs).block_until_ready()
# for #R 1000 and 115 nbrs
# without checkpoint 268ms
# with checkpoint 397ms 7 runs 1 loop

1.58 s ± 366 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
from uf3.jax.potentials import uf3_mapped

In [23]:
f = uf3_mapped(knots3)
c = xla_computation(f)(dR,dR,coefficients=coefficients3)

In [ ]:
print(c.as_hlo_text())

In [ ]:
displacement, shift = space.periodic(side_length)

init_fn, apply_fn = simulate.nve(energy_fn, shift, 1e-3)
state = init_fn(random.PRNGKey(0), R, kT=1e-3, neighbor=nbrs)


def body_fn(i, state):
    state, nbrs = state
    nbrs = nbrs.update(state.position)
    state = apply_fn(state, neighbor=nbrs)
    return state, nbrs


print('Step\tKE\tPE\tTotal Energy\ttime/step')
print('----------------------------------------')
PE = []
KE = []
print_every = 4
step = 0
old_time = time.time_ns()
while step < 40:
    new_state, nbrs = lax.fori_loop(0, 100, body_fn, (state, nbrs))
    if nbrs.did_buffer_overflow:
        print("Neighbor list overflowed, reallocating.")
        nbrs = neighbor_fn.allocate(state.position)
    else:
        state = new_state
        step += 1

    if step % print_every == 0 and step > 0:
        new_time = time.time_ns()
        PE += [energy_fn(state.position, nbrs)]
        KE += [quantity.kinetic_energy(state.velocity)]
        print(
            "{}\t{:.2f}\t{:.2f}\t{:.2f}\t\t{:.2f}ms".format(
                step * print_every,
                KE[-1],
                PE[-1],
                KE[-1] + PE[-1],
                (new_time - old_time) / print_every / 100.0 / 1000000.0,
            )
        )
        old_time = new_time

Step	KE	PE	Total Energy	time/step
----------------------------------------
Neighbor list overflowed, reallocating.
16	97844.10	-32208.65	65635.45		537.55ms
Neighbor list overflowed, reallocating.
32	350235.56	14729.73	364965.29		517.95ms
